![Logo BV IBMEC](https://raw.githubusercontent.com/ian-iania/IBMEC-BV-Modelos-Preditivos/main/logo-bv-ibmec-notebooks.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ian-iania/IBMEC-BV-Modelos-Preditivos/blob/main/notebooks/NB01_Regression.ipynb)

# NB01 - Regressão e Séries Temporais (FP&A Banco BV)

**Objetivo:** aplicar modelos de regressão e séries temporais em dados sintéticos de FP&A do BV, com linguagem simples e foco em negócio.
**Neste notebook, vamos comparar baseline e modelos usando split temporal e métricas MAE/RMSE.**
**A ideia é entender o processo de ponta a ponta, sem exigir experiência prévia em programação.**


## 1) Setup do notebook

### O que vamos fazer
Importar bibliotecas usadas na aula.

### Por que isso importa para FP&A
Essas bibliotecas permitem carregar dados, calcular métricas e visualizar resultados.

### O que você deve ver no output
Sem erro de execução.


In [ ]:
import pandas as pd  # biblioteca para carregar e manipular tabelas (DataFrames)
import numpy as np  # biblioteca para operações numéricas
import matplotlib.pyplot as plt  # biblioteca para criar gráficos
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet  # modelos lineares
from sklearn.tree import DecisionTreeRegressor  # modelo de árvore de decisão
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor  # modelos de ensemble baseados em árvores
from sklearn.metrics import mean_absolute_error, mean_squared_error  # métricas de avaliação MAE e RMSE
from statsmodels.tsa.holtwinters import ExponentialSmoothing  # modelo ETS (Holt-Winters)
from statsmodels.tsa.statespace.sarimax import SARIMAX  # modelo ARIMA/ARIMAX


## 2) PARTE A - Regressão com dataset linear (X -> y)

### O que vamos fazer
Carregar o dataset linear e construir modelos que explicam `y` a partir de drivers (`X`).

### Por que isso importa para FP&A
Esse formato ajuda a responder: "quanto a originação muda se um driver mudar?".


### 2.1 Carregar dados prontos

Os datasets já foram gerados e estão prontos para uso. Aqui vamos carregar direto do GitHub.


In [ ]:
url_linear = 'https://raw.githubusercontent.com/ian-iania/IBMEC-BV-Modelos-Preditivos/main/data/bv_fpa_regressao_linear.csv'  # URL do dataset linear
df_lin = pd.read_csv(url_linear)  # leitura do CSV para DataFrame
df_lin['ds'] = pd.to_datetime(df_lin['ds'])  # conversão da coluna de data para tipo datetime
df_lin.head()  # visualização das primeiras linhas


### 2.2 Dicionário das colunas (muito importante)

- `ds`: data de referência mensal (início do mês)
- `y`: originação mensal (R$ milhões) -> **variável alvo**
- `selic`: taxa SELIC
- `desemprego`: taxa de desemprego
- `ltv_medio`: Loan-to-Value médio da carteira
- `spread`: spread médio (relacionado à SELIC)
- `marketing`: esforço/investimento de marketing (índice sintético)
- `mix_auto`: participação do mix auto (0 a 1)
- `mes`: mês numérico (1 a 12), usado para sazonalidade

Se quiser pensar em nomes mais "de negócio":
- `ds` = `data_referencia`
- `y` = `originacao_mensal_m`

Neste notebook manteremos `ds` e `y` porque esses nomes são padrão em modelos como Prophet.


In [ ]:
print('shape:', df_lin.shape)  # exibe quantidade de linhas e colunas
print('colunas:', list(df_lin.columns))  # lista todas as colunas da base
print('tipos:')  # título visual para facilitar leitura
print(df_lin.dtypes)  # mostra o tipo de cada coluna


### 2.3 Visualização inicial

### O que vamos fazer
Ver comportamento temporal de `y` e relação simples entre `selic` e `y`.

### O que você deve ver no output
Um gráfico de linha e um gráfico de dispersão.


In [ ]:
plt.figure(figsize=(10, 4))  # cria a área do gráfico com tamanho definido
plt.plot(df_lin['ds'], df_lin['y'], linewidth=2)  # plota y ao longo do tempo
plt.title('PARTE A - Originação (y) ao longo do tempo')  # define o título do gráfico
plt.xlabel('Data')  # define o rótulo do eixo X
plt.ylabel('y (R$ milhões)')  # define o rótulo do eixo Y
plt.grid(alpha=0.3)  # adiciona grade para facilitar leitura
plt.show()  # renderiza o gráfico na saída


In [ ]:
plt.figure(figsize=(6, 4))  # cria a área do gráfico de dispersão
plt.scatter(df_lin['selic'], df_lin['y'], alpha=0.75)  # plota pontos SELIC x y
plt.title('PARTE A - SELIC vs y')  # define o título do gráfico
plt.xlabel('SELIC')  # define o rótulo do eixo X
plt.ylabel('y (R$ milhões)')  # define o rótulo do eixo Y
plt.grid(alpha=0.3)  # adiciona grade para facilitar leitura
plt.show()  # renderiza o gráfico na saída


### 2.4 Conceito de split temporal (sem embaralhar)

### O que vamos fazer
Separar treino e teste por tempo (80/20):
- Treino: período mais antigo
- Teste: período mais recente

### Por que isso importa para FP&A
No mundo real, sempre prevemos o futuro com base no passado.


In [ ]:
features = ['selic', 'desemprego', 'ltv_medio', 'spread', 'marketing', 'mix_auto', 'mes']  # lista de variáveis explicativas
split_idx = int(len(df_lin) * 0.8)  # calcula ponto de corte de 80% para treino
train_lin = df_lin.iloc[:split_idx].copy()  # recorte do treino (parte inicial)
test_lin = df_lin.iloc[split_idx:].copy()  # recorte do teste (parte final)
X_train_lin = train_lin[features]  # matriz de features no treino
X_test_lin = test_lin[features]  # matriz de features no teste
y_train_lin = train_lin['y']  # vetor alvo no treino
y_test_lin = test_lin['y']  # vetor alvo no teste
print('Treino:', X_train_lin.shape, 'Teste:', X_test_lin.shape)  # exibe tamanhos das matrizes
print('Período treino:', train_lin['ds'].min().date(), '->', train_lin['ds'].max().date())  # exibe janela temporal de treino
print('Período teste :', test_lin['ds'].min().date(), '->', test_lin['ds'].max().date())  # exibe janela temporal de teste


### 2.5 Baseline (referência mínima)

### O que vamos fazer
Criar baseline naive: repetir no teste o último valor observado no treino.

### Por que isso importa para FP&A
Um modelo só vale a pena se superar baseline.


In [ ]:
baseline_value_lin = y_train_lin.iloc[-1]  # captura último valor de y no treino
pred_baseline_lin = np.repeat(baseline_value_lin, len(y_test_lin))  # repete o último valor para todos os meses do teste
mae_baseline_lin = mean_absolute_error(y_test_lin, pred_baseline_lin)  # calcula MAE do baseline
rmse_baseline_lin = np.sqrt(mean_squared_error(y_test_lin, pred_baseline_lin))  # calcula RMSE do baseline
print('Baseline MAE :', round(mae_baseline_lin, 3))  # imprime MAE arredondado
print('Baseline RMSE:', round(rmse_baseline_lin, 3))  # imprime RMSE arredondado


### 2.6 Modelos de regressão (execução linear)

Vamos treinar um modelo por célula, de forma didática.


In [ ]:
model_lr = LinearRegression()  # cria modelo de regressão linear
model_lr.fit(X_train_lin, y_train_lin)  # treina modelo com dados de treino
pred_lr = model_lr.predict(X_test_lin)  # gera previsões para o teste
mae_lr = mean_absolute_error(y_test_lin, pred_lr)  # calcula MAE da regressão linear
rmse_lr = np.sqrt(mean_squared_error(y_test_lin, pred_lr))  # calcula RMSE da regressão linear
print('LinearRegression RMSE:', round(rmse_lr, 3))  # imprime RMSE para consulta rápida


In [ ]:
model_ridge = Ridge(alpha=1.0, random_state=42)  # cria modelo Ridge com parâmetro alpha
model_ridge.fit(X_train_lin, y_train_lin)  # treina modelo Ridge
pred_ridge = model_ridge.predict(X_test_lin)  # gera previsões no teste
mae_ridge = mean_absolute_error(y_test_lin, pred_ridge)  # calcula MAE do Ridge
rmse_ridge = np.sqrt(mean_squared_error(y_test_lin, pred_ridge))  # calcula RMSE do Ridge
print('Ridge RMSE:', round(rmse_ridge, 3))  # imprime RMSE para consulta rápida


In [ ]:
model_en = ElasticNet(alpha=0.05, l1_ratio=0.5, random_state=42, max_iter=5000)  # cria modelo ElasticNet
model_en.fit(X_train_lin, y_train_lin)  # treina modelo ElasticNet
pred_en = model_en.predict(X_test_lin)  # gera previsões no teste
mae_en = mean_absolute_error(y_test_lin, pred_en)  # calcula MAE do ElasticNet
rmse_en = np.sqrt(mean_squared_error(y_test_lin, pred_en))  # calcula RMSE do ElasticNet
print('ElasticNet RMSE:', round(rmse_en, 3))  # imprime RMSE para consulta rápida


In [ ]:
model_tree = DecisionTreeRegressor(max_depth=4, random_state=42)  # cria árvore de decisão com profundidade controlada
model_tree.fit(X_train_lin, y_train_lin)  # treina árvore de decisão
pred_tree = model_tree.predict(X_test_lin)  # gera previsões no teste
mae_tree = mean_absolute_error(y_test_lin, pred_tree)  # calcula MAE da árvore
rmse_tree = np.sqrt(mean_squared_error(y_test_lin, pred_tree))  # calcula RMSE da árvore
print('DecisionTree RMSE:', round(rmse_tree, 3))  # imprime RMSE para consulta rápida


In [ ]:
model_rf = RandomForestRegressor(n_estimators=300, max_depth=6, random_state=42)  # cria Random Forest com 300 árvores
model_rf.fit(X_train_lin, y_train_lin)  # treina Random Forest
pred_rf = model_rf.predict(X_test_lin)  # gera previsões no teste
mae_rf = mean_absolute_error(y_test_lin, pred_rf)  # calcula MAE do Random Forest
rmse_rf = np.sqrt(mean_squared_error(y_test_lin, pred_rf))  # calcula RMSE do Random Forest
print('RandomForest RMSE:', round(rmse_rf, 3))  # imprime RMSE para consulta rápida


In [ ]:
model_gbm = GradientBoostingRegressor(random_state=42)  # cria Gradient Boosting com semente fixa
model_gbm.fit(X_train_lin, y_train_lin)  # treina Gradient Boosting
pred_gbm = model_gbm.predict(X_test_lin)  # gera previsões no teste
mae_gbm = mean_absolute_error(y_test_lin, pred_gbm)  # calcula MAE do Gradient Boosting
rmse_gbm = np.sqrt(mean_squared_error(y_test_lin, pred_gbm))  # calcula RMSE do Gradient Boosting
print('GradientBoosting RMSE:', round(rmse_gbm, 3))  # imprime RMSE para consulta rápida


In [ ]:
results_a = pd.DataFrame([  # cria tabela de resultados da PARTE A
    {'modelo': 'Baseline_Naive', 'mae': mae_baseline_lin, 'rmse': rmse_baseline_lin},  # linha do baseline
    {'modelo': 'LinearRegression', 'mae': mae_lr, 'rmse': rmse_lr},  # linha do LinearRegression
    {'modelo': 'Ridge', 'mae': mae_ridge, 'rmse': rmse_ridge},  # linha do Ridge
    {'modelo': 'ElasticNet', 'mae': mae_en, 'rmse': rmse_en},  # linha do ElasticNet
    {'modelo': 'DecisionTree', 'mae': mae_tree, 'rmse': rmse_tree},  # linha da árvore
    {'modelo': 'RandomForest', 'mae': mae_rf, 'rmse': rmse_rf},  # linha do RandomForest
    {'modelo': 'GradientBoosting', 'mae': mae_gbm, 'rmse': rmse_gbm},  # linha do GradientBoosting
]).sort_values('rmse').reset_index(drop=True)  # ordena por melhor RMSE
results_a  # exibe tabela final da PARTE A


In [ ]:
best_linear_name = results_a[results_a['modelo'].isin(['LinearRegression', 'Ridge', 'ElasticNet'])].iloc[0]['modelo']  # identifica melhor linear
best_nonlinear_name = results_a[results_a['modelo'].isin(['DecisionTree', 'RandomForest', 'GradientBoosting'])].iloc[0]['modelo']  # identifica melhor não-linear
pred_map_a = {  # mapeia nome do modelo para seu vetor de previsão
    'LinearRegression': pred_lr,  # previsões do LinearRegression
    'Ridge': pred_ridge,  # previsões do Ridge
    'ElasticNet': pred_en,  # previsões do ElasticNet
    'DecisionTree': pred_tree,  # previsões da árvore
    'RandomForest': pred_rf,  # previsões do RandomForest
    'GradientBoosting': pred_gbm,  # previsões do GradientBoosting
}  # fim do dicionário de previsões
plt.figure(figsize=(11, 4))  # cria área do gráfico comparativo
plt.plot(test_lin['ds'], y_test_lin.values, label='Real', color='black', linewidth=2)  # plota série real
plt.plot(test_lin['ds'], pred_baseline_lin, label='Baseline_Naive', linewidth=1.8)  # plota baseline
plt.plot(test_lin['ds'], pred_map_a[best_linear_name], label=f'Melhor linear: {best_linear_name}', linewidth=1.8)  # plota melhor linear
plt.plot(test_lin['ds'], pred_map_a[best_nonlinear_name], label=f'Melhor não-linear: {best_nonlinear_name}', linewidth=1.8)  # plota melhor não-linear
plt.title('PARTE A - Real vs Previsto (teste)')  # define título do gráfico
plt.xlabel('Data')  # define rótulo do eixo X
plt.ylabel('y (R$ milhões)')  # define rótulo do eixo Y
plt.legend()  # exibe legenda
plt.grid(alpha=0.3)  # adiciona grade
plt.show()  # renderiza gráfico


### 2.7 Feature importance

### O que vamos fazer
Mostrar importância das variáveis em modelos de árvore.

### Observação importante
Importância ajuda na leitura, mas **não significa causalidade**.


In [ ]:
rf_imp = pd.Series(model_rf.feature_importances_, index=features).sort_values(ascending=True)  # calcula importâncias do RandomForest
gbm_imp = pd.Series(model_gbm.feature_importances_, index=features).sort_values(ascending=True)  # calcula importâncias do GradientBoosting
plt.figure(figsize=(12, 4))  # cria figura com dois gráficos lado a lado
plt.subplot(1, 2, 1)  # ativa painel 1
plt.barh(rf_imp.index, rf_imp.values)  # plota barras horizontais do RandomForest
plt.title('Importância - RandomForest')  # define título do painel 1
plt.grid(axis='x', alpha=0.3)  # adiciona grade horizontal no painel 1
plt.subplot(1, 2, 2)  # ativa painel 2
plt.barh(gbm_imp.index, gbm_imp.values)  # plota barras horizontais do GradientBoosting
plt.title('Importância - GradientBoosting')  # define título do painel 2
plt.grid(axis='x', alpha=0.3)  # adiciona grade horizontal no painel 2
plt.tight_layout()  # ajusta espaçamento entre painéis
plt.show()  # renderiza os dois painéis


## 3) PARTE B - Mesmo pipeline no dataset não-linear

### O que vamos fazer
Repetir o mesmo fluxo no dataset com não-linearidades.

### Por que isso importa para FP&A
Assim vemos quando modelos não-lineares capturam melhor os padrões.


In [ ]:
url_nlin = 'https://raw.githubusercontent.com/ian-iania/IBMEC-BV-Modelos-Preditivos/main/data/bv_fpa_regressao_nonlinear.csv'  # URL do dataset não-linear
df_nlin = pd.read_csv(url_nlin)  # leitura do CSV não-linear
df_nlin['ds'] = pd.to_datetime(df_nlin['ds'])  # conversão da data para datetime
df_nlin.head()  # visualização inicial do dataset não-linear


In [ ]:
split_idx_b = int(len(df_nlin) * 0.8)  # calcula ponto de corte de 80% para treino
train_nlin = df_nlin.iloc[:split_idx_b].copy()  # separa período de treino
train_nlin = train_nlin.sort_values('ds')  # garante ordenação temporal no treino
test_nlin = df_nlin.iloc[split_idx_b:].copy()  # separa período de teste
test_nlin = test_nlin.sort_values('ds')  # garante ordenação temporal no teste
X_train_nlin = train_nlin[features]  # extrai features de treino
X_test_nlin = test_nlin[features]  # extrai features de teste
y_train_nlin = train_nlin['y']  # extrai alvo de treino
y_test_nlin = test_nlin['y']  # extrai alvo de teste
baseline_value_nlin = y_train_nlin.iloc[-1]  # captura último valor de treino para baseline
pred_baseline_nlin = np.repeat(baseline_value_nlin, len(y_test_nlin))  # monta previsão constante do baseline
mae_baseline_nlin = mean_absolute_error(y_test_nlin, pred_baseline_nlin)  # calcula MAE do baseline não-linear
rmse_baseline_nlin = np.sqrt(mean_squared_error(y_test_nlin, pred_baseline_nlin))  # calcula RMSE do baseline não-linear
print('Baseline RMSE (não-linear):', round(rmse_baseline_nlin, 3))  # imprime RMSE do baseline da PARTE B


In [ ]:
model_lr_b = LinearRegression()  # cria LinearRegression da PARTE B
model_lr_b.fit(X_train_nlin, y_train_nlin)  # treina LinearRegression na base não-linear
pred_lr_b = model_lr_b.predict(X_test_nlin)  # gera previsões LinearRegression na PARTE B
model_ridge_b = Ridge(alpha=1.0, random_state=42)  # cria Ridge da PARTE B
model_ridge_b.fit(X_train_nlin, y_train_nlin)  # treina Ridge na base não-linear
pred_ridge_b = model_ridge_b.predict(X_test_nlin)  # gera previsões Ridge na PARTE B
model_en_b = ElasticNet(alpha=0.05, l1_ratio=0.5, random_state=42, max_iter=5000)  # cria ElasticNet da PARTE B
model_en_b.fit(X_train_nlin, y_train_nlin)  # treina ElasticNet na base não-linear
pred_en_b = model_en_b.predict(X_test_nlin)  # gera previsões ElasticNet na PARTE B
model_tree_b = DecisionTreeRegressor(max_depth=4, random_state=42)  # cria árvore da PARTE B
model_tree_b.fit(X_train_nlin, y_train_nlin)  # treina árvore na base não-linear
pred_tree_b = model_tree_b.predict(X_test_nlin)  # gera previsões da árvore na PARTE B
model_rf_b = RandomForestRegressor(n_estimators=300, max_depth=6, random_state=42)  # cria RandomForest da PARTE B
model_rf_b.fit(X_train_nlin, y_train_nlin)  # treina RandomForest na base não-linear
pred_rf_b = model_rf_b.predict(X_test_nlin)  # gera previsões RandomForest na PARTE B
model_gbm_b = GradientBoostingRegressor(random_state=42)  # cria GradientBoosting da PARTE B
model_gbm_b.fit(X_train_nlin, y_train_nlin)  # treina GradientBoosting na base não-linear
pred_gbm_b = model_gbm_b.predict(X_test_nlin)  # gera previsões GradientBoosting na PARTE B


In [ ]:
results_b = pd.DataFrame([  # cria tabela de resultados da PARTE B
    {'modelo': 'Baseline_Naive', 'mae': mae_baseline_nlin, 'rmse': rmse_baseline_nlin},  # linha baseline
    {'modelo': 'LinearRegression', 'mae': mean_absolute_error(y_test_nlin, pred_lr_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_lr_b))},  # linha linear
    {'modelo': 'Ridge', 'mae': mean_absolute_error(y_test_nlin, pred_ridge_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_ridge_b))},  # linha ridge
    {'modelo': 'ElasticNet', 'mae': mean_absolute_error(y_test_nlin, pred_en_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_en_b))},  # linha elasticnet
    {'modelo': 'DecisionTree', 'mae': mean_absolute_error(y_test_nlin, pred_tree_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_tree_b))},  # linha árvore
    {'modelo': 'RandomForest', 'mae': mean_absolute_error(y_test_nlin, pred_rf_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_rf_b))},  # linha random forest
    {'modelo': 'GradientBoosting', 'mae': mean_absolute_error(y_test_nlin, pred_gbm_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_gbm_b))},  # linha gradient boosting
]).sort_values('rmse').reset_index(drop=True)  # ordena por melhor RMSE
results_b  # exibe ranking final da PARTE B


In [ ]:
best_linear_b = results_b[results_b['modelo'].isin(['LinearRegression', 'Ridge', 'ElasticNet'])].iloc[0]['modelo']  # identifica melhor modelo linear da PARTE B
best_nonlinear_b = results_b[results_b['modelo'].isin(['DecisionTree', 'RandomForest', 'GradientBoosting'])].iloc[0]['modelo']  # identifica melhor modelo não-linear da PARTE B
pred_map_b = {  # mapeia nome para vetor de previsão na PARTE B
    'LinearRegression': pred_lr_b,  # previsões linear
    'Ridge': pred_ridge_b,  # previsões ridge
    'ElasticNet': pred_en_b,  # previsões elasticnet
    'DecisionTree': pred_tree_b,  # previsões árvore
    'RandomForest': pred_rf_b,  # previsões random forest
    'GradientBoosting': pred_gbm_b,  # previsões gradient boosting
}  # fim do dicionário de previsões da PARTE B
plt.figure(figsize=(11, 4))  # cria área do gráfico comparativo da PARTE B
plt.plot(test_nlin['ds'], y_test_nlin.values, label='Real', color='black', linewidth=2)  # plota série real da PARTE B
plt.plot(test_nlin['ds'], pred_map_b[best_linear_b], label=f'Melhor linear: {best_linear_b}', linewidth=1.8)  # plota melhor linear da PARTE B
plt.plot(test_nlin['ds'], pred_map_b[best_nonlinear_b], label=f'Melhor não-linear: {best_nonlinear_b}', linewidth=1.8)  # plota melhor não-linear da PARTE B
plt.title('PARTE B - Melhor linear vs melhor não-linear (teste)')  # define título do gráfico da PARTE B
plt.xlabel('Data')  # define rótulo do eixo X na PARTE B
plt.ylabel('y (R$ milhões)')  # define rótulo do eixo Y na PARTE B
plt.legend()  # exibe legenda da PARTE B
plt.grid(alpha=0.3)  # adiciona grade da PARTE B
plt.show()  # renderiza gráfico da PARTE B


## 4) PARTE C - Séries temporais (t -> y)

### O que vamos fazer
Comparar baseline, ETS, ARIMA, ARIMAX e Prophet (opcional).

### Por que isso importa para FP&A
Em vários contextos, o próprio histórico de `y` já é forte para previsão.


In [ ]:
url_ts = 'https://raw.githubusercontent.com/ian-iania/IBMEC-BV-Modelos-Preditivos/main/data/bv_fpa_timeseries.csv'  # URL do dataset de séries temporais
df_ts = pd.read_csv(url_ts)  # leitura do CSV de séries temporais
df_ts['ds'] = pd.to_datetime(df_ts['ds'])  # conversão da data para datetime
df_ts.head()  # visualização inicial do dataset temporal


In [ ]:
plt.figure(figsize=(10, 4))  # cria figura para gráfico temporal
plt.plot(df_ts['ds'], df_ts['y'], linewidth=2)  # plota y ao longo do tempo
plt.title('PARTE C - Série temporal y')  # define título do gráfico temporal
plt.xlabel('Data')  # define eixo X no gráfico temporal
plt.ylabel('y (R$ milhões)')  # define eixo Y no gráfico temporal
plt.grid(alpha=0.3)  # adiciona grade no gráfico temporal
plt.show()  # renderiza gráfico temporal


In [ ]:
holdout = 12  # define os últimos 12 meses como teste
train_ts = df_ts.iloc[:-holdout].copy()  # separa parte inicial da série para treino
test_ts = df_ts.iloc[-holdout:].copy()  # separa parte final da série para teste
y_train_ts = train_ts['y'].values  # vetor alvo de treino em formato numpy
y_test_ts = test_ts['y'].values  # vetor alvo de teste em formato numpy
print('Treino:', train_ts.shape, 'Teste:', test_ts.shape)  # imprime tamanhos de treino e teste
print('Período treino:', train_ts['ds'].min().date(), '->', train_ts['ds'].max().date())  # imprime janela temporal de treino
print('Período teste :', test_ts['ds'].min().date(), '->', test_ts['ds'].max().date())  # imprime janela temporal de teste


In [ ]:
pred_naive_ts = np.repeat(y_train_ts[-1], len(y_test_ts))  # cria baseline naive repetindo último valor do treino
mae_naive_ts = mean_absolute_error(y_test_ts, pred_naive_ts)  # calcula MAE do baseline temporal
rmse_naive_ts = np.sqrt(mean_squared_error(y_test_ts, pred_naive_ts))  # calcula RMSE do baseline temporal
print('Baseline MAE :', round(mae_naive_ts, 3))  # imprime MAE do baseline temporal
print('Baseline RMSE:', round(rmse_naive_ts, 3))  # imprime RMSE do baseline temporal


In [ ]:
ets = ExponentialSmoothing(train_ts['y'], trend='add', seasonal='add', seasonal_periods=12).fit(optimized=True)  # ajusta modelo ETS
pred_ets = ets.forecast(len(test_ts)).values  # gera previsão ETS para o período de teste
mae_ets = mean_absolute_error(y_test_ts, pred_ets)  # calcula MAE do ETS
rmse_ets = np.sqrt(mean_squared_error(y_test_ts, pred_ets))  # calcula RMSE do ETS
print('ETS MAE :', round(mae_ets, 3))  # imprime MAE do ETS
print('ETS RMSE:', round(rmse_ets, 3))  # imprime RMSE do ETS


In [ ]:
arima = SARIMAX(train_ts['y'], order=(1, 1, 1), enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)  # ajusta modelo ARIMA
pred_arima = arima.forecast(steps=len(test_ts)).values  # gera previsão ARIMA para o teste
mae_arima = mean_absolute_error(y_test_ts, pred_arima)  # calcula MAE do ARIMA
rmse_arima = np.sqrt(mean_squared_error(y_test_ts, pred_arima))  # calcula RMSE do ARIMA
print('ARIMA MAE :', round(mae_arima, 3))  # imprime MAE do ARIMA
print('ARIMA RMSE:', round(rmse_arima, 3))  # imprime RMSE do ARIMA


In [ ]:
arimax = SARIMAX(train_ts['y'], exog=train_ts[['selic']], order=(1, 1, 1), enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)  # ajusta ARIMAX com selic
pred_arimax = arimax.forecast(steps=len(test_ts), exog=test_ts[['selic']]).values  # gera previsão ARIMAX usando selic no teste
mae_arimax = mean_absolute_error(y_test_ts, pred_arimax)  # calcula MAE do ARIMAX
rmse_arimax = np.sqrt(mean_squared_error(y_test_ts, pred_arimax))  # calcula RMSE do ARIMAX
print('ARIMAX MAE :', round(mae_arimax, 3))  # imprime MAE do ARIMAX
print('ARIMAX RMSE:', round(rmse_arimax, 3))  # imprime RMSE do ARIMAX


### 4.1 Prophet (opcional)

Prophet pode demorar para instalar em algumas sessões do Colab.
Se não funcionar, continue a aula normalmente com ETS/ARIMA/ARIMAX.


In [ ]:
import subprocess  # módulo para executar comando de instalação
import sys  # módulo para descobrir o executável Python atual
prophet_ok = False  # flag para indicar disponibilidade do Prophet
try:  # tenta importar Prophet direto
    from prophet import Prophet  # importa classe Prophet se já estiver instalada
    prophet_ok = True  # marca que Prophet está disponível
    print('Prophet já disponível.')  # informa disponibilidade imediata
except Exception:  # captura erro de importação
    print('Tentando instalar Prophet...')  # avisa tentativa de instalação
    try:  # tenta instalar via pip
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'prophet', '-q'])  # executa instalação silenciosa
        from prophet import Prophet  # importa novamente após instalar
        prophet_ok = True  # marca sucesso da instalação
        print('Prophet instalado com sucesso.')  # confirma instalação
    except Exception as e:  # captura erro na instalação
        print('Prophet indisponível nesta sessão.')  # avisa indisponibilidade
        print('Erro:', e)  # mostra detalhe do erro


In [ ]:
pred_prophet = None  # inicializa previsão do Prophet como vazia
mae_prophet = None  # inicializa MAE do Prophet como vazio
rmse_prophet = None  # inicializa RMSE do Prophet como vazio
if prophet_ok:  # executa bloco apenas se Prophet estiver disponível
    train_prophet = train_ts[['ds', 'y', 'selic', 'evento']].copy()  # cria base de treino no formato Prophet
    test_prophet = test_ts[['ds', 'selic', 'evento']].copy()  # cria base de teste no formato Prophet
    m = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)  # cria modelo Prophet com sazonalidade anual
    m.add_regressor('selic')  # adiciona selic como regressora adicional
    m.add_regressor('evento')  # adiciona evento como regressora adicional
    m.fit(train_prophet)  # treina Prophet com dados de treino
    fcst = m.predict(test_prophet)  # gera previsão para período de teste
    pred_prophet = fcst['yhat'].values  # extrai vetor previsto do Prophet
    mae_prophet = mean_absolute_error(y_test_ts, pred_prophet)  # calcula MAE do Prophet
    rmse_prophet = np.sqrt(mean_squared_error(y_test_ts, pred_prophet))  # calcula RMSE do Prophet
    print('Prophet MAE :', round(mae_prophet, 3))  # imprime MAE do Prophet
    print('Prophet RMSE:', round(rmse_prophet, 3))  # imprime RMSE do Prophet
else:  # executa bloco alternativo se Prophet não estiver disponível
    print('Prophet não executado.')  # informa que etapa foi pulada


In [ ]:
results_c = pd.DataFrame([  # cria tabela base de resultados da PARTE C
    {'modelo': 'Baseline_Naive', 'mae': mae_naive_ts, 'rmse': rmse_naive_ts},  # linha baseline
    {'modelo': 'ETS_HoltWinters', 'mae': mae_ets, 'rmse': rmse_ets},  # linha ETS
    {'modelo': 'ARIMA', 'mae': mae_arima, 'rmse': rmse_arima},  # linha ARIMA
    {'modelo': 'ARIMAX_selic', 'mae': mae_arimax, 'rmse': rmse_arimax},  # linha ARIMAX
])  # fecha criação inicial do DataFrame
if pred_prophet is not None:  # verifica se Prophet foi executado
    results_c = pd.concat([results_c, pd.DataFrame([{'modelo': 'Prophet', 'mae': mae_prophet, 'rmse': rmse_prophet}])], ignore_index=True)  # adiciona linha do Prophet
results_c = results_c.sort_values('rmse').reset_index(drop=True)  # ordena tabela final por RMSE
results_c  # exibe ranking final da PARTE C


In [ ]:
plt.figure(figsize=(11, 4))  # cria figura do gráfico final da PARTE C
plt.plot(test_ts['ds'], y_test_ts, label='Real', color='black', linewidth=2)  # plota série real do teste
plt.plot(test_ts['ds'], pred_naive_ts, label='Baseline_Naive', linewidth=1.8)  # plota baseline naive
plt.plot(test_ts['ds'], pred_ets, label='ETS_HoltWinters', linewidth=1.8)  # plota previsão ETS
plt.plot(test_ts['ds'], pred_arimax, label='ARIMAX_selic', linewidth=1.8)  # plota previsão ARIMAX
if pred_prophet is not None:  # verifica se há previsão do Prophet
    plt.plot(test_ts['ds'], pred_prophet, label='Prophet', linewidth=1.8)  # plota previsão Prophet quando disponível
plt.title('PARTE C - Real vs Previsões (teste)')  # define título do gráfico final
plt.xlabel('Data')  # define rótulo do eixo X no gráfico final
plt.ylabel('y (R$ milhões)')  # define rótulo do eixo Y no gráfico final
plt.legend()  # exibe legenda do gráfico final
plt.grid(alpha=0.3)  # adiciona grade do gráfico final
plt.show()  # renderiza gráfico final da PARTE C


## 5) Conclusão didática

- **X -> y (regressão):** útil quando temos drivers para explicar comportamento da originação.
- **t -> y (série temporal):** útil quando o histórico de `y` já carrega padrão forte.
- **Regra de ouro:** comparar com baseline e validar no tempo.


## 6) Checklist FP&A

- Baseline: sempre comparar modelo novo com referência simples.
- Split temporal: nunca misturar passado e futuro na validação.
- Métricas: usar MAE e RMSE no teste para decidir modelo.
- Governança: documentar dados, período e versão do modelo.
- Monitoramento: revisar performance regularmente e recalibrar quando necessário.
